# 1. 라이브러리 설치

In [1]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


# 2. 클래스 자동 추출 및 XML 변환, 데이터 분할

In [5]:
import os
import xml.etree.ElementTree as ET
import shutil
import random
from pathlib import Path

# 프로젝트 루트 디렉토리 (현재 노트북 파일이 위치한 곳)
project_root = Path.cwd()

# 데이터셋 폴더 경로 (상대 경로 사용)
xml_folder = project_root / 'New_sample' / '라벨링데이터'
image_folder = project_root / 'New_sample' / '원천데이터'
dataset_folder = project_root / 'dataset'
labels_all = dataset_folder / 'labels_all'  # 모든 라벨을 저장할 폴더
labels_train = dataset_folder / 'labels' / 'train'
labels_val = dataset_folder / 'labels' / 'val'
images_train = dataset_folder / 'images' / 'train'
images_val = dataset_folder / 'images' / 'val'

# 데이터셋 폴더 생성
for folder in [labels_all, labels_train, labels_val, images_train, images_val]:
    folder.mkdir(parents=True, exist_ok=True)

# 클래스 이름 자동 추출 (원천데이터 폴더의 하위 폴더 이름 사용)
# 폴더 이름에서 클래스 이름만 추출 (예: '10060_해태포키블루베리41G' -> '해태포키블루베리41G')
classes_full = sorted([folder.name for folder in image_folder.iterdir() if folder.is_dir()])
classes = [cls.split('_', 1)[1] if '_' in cls else cls for cls in classes_full]
class_map = {cls: idx for idx, cls in enumerate(classes)}

print("클래스 목록:", class_map)

def convert_xml_to_yolo(xml_folder, image_folder, label_folder, class_map):
    for xml_file in xml_folder.rglob('*.xml'):
        if '_meta' in xml_file.name:
            continue  # meta.xml 파일 무시

        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            filename = root.find('filename').text
            # 이미지 파일 경로
            image_path = image_folder / root.find('filename').text

            # 이미지 크기 가져오기
            size = root.find('size')
            width = float(size.find('width').text)
            height = float(size.find('height').text)

            yolo_labels = []

            for obj in root.findall('object'):
                cls_name = obj.find('name').text
                if cls_name not in class_map:
                    print(f"클래스 '{cls_name}'가 class_map에 없습니다. 무시합니다.")
                    continue  # 정의되지 않은 클래스 무시
                cls_id = class_map[cls_name]

                bndbox = obj.find('bndbox')
                xmin = float(bndbox.find('xmin').text)
                ymin = float(bndbox.find('ymin').text)
                xmax = float(bndbox.find('xmax').text)
                ymax = float(bndbox.find('ymax').text)

                # YOLO 좌표 계산 (정규화)
                x_center = ((xmin + xmax) / 2) / width
                y_center = ((ymin + ymax) / 2) / height
                box_width = (xmax - xmin) / width
                box_height = (ymax - ymin) / height

                yolo_labels.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

            # YOLO 라벨 파일 저장
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_path = label_folder / label_filename
            with open(label_path, 'w') as f:
                for label in yolo_labels:
                    f.write(label + '\n')
        except ET.ParseError as e:
            print(f"XML 파싱 오류: {xml_file} - {e}")

def split_dataset(image_folder, label_folder_all, output_image_train, output_image_val, output_label_train, output_label_val, split_ratio=0.8):
    images = sorted([f for f in image_folder.glob('**/*') if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']])
    random.shuffle(images)
    split_idx = int(len(images) * split_ratio)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    for img_path in train_images:
        relative_path = img_path.relative_to(image_folder)
        destination_img_path = output_image_train / relative_path.name
        destination_label_path = output_label_train / (relative_path.stem + '.txt')
        shutil.copy(img_path, destination_img_path)
        label_src = label_folder_all / (relative_path.stem + '.txt')
        if label_src.exists():
            shutil.copy(label_src, destination_label_path)
        else:
            print(f"라벨 파일이 존재하지 않습니다: {label_src}")

    for img_path in val_images:
        relative_path = img_path.relative_to(image_folder)
        destination_img_path = output_image_val / relative_path.name
        destination_label_path = output_label_val / (relative_path.stem + '.txt')
        shutil.copy(img_path, destination_img_path)
        label_src = label_folder_all / (relative_path.stem + '.txt')
        if label_src.exists():
            shutil.copy(label_src, destination_label_path)
        else:
            print(f"라벨 파일이 존재하지 않습니다: {label_src}")

# XML을 YOLO 형식으로 변환하여 labels_all 폴더에 저장
convert_xml_to_yolo(xml_folder, image_folder, labels_all, class_map)

# 데이터 분할 (훈련:검증 = 80:20)
split_dataset(image_folder, labels_all, images_train, images_val, labels_train, labels_val, split_ratio=0.8)

print("XML to YOLO 변환 및 데이터 분할 완료!")


클래스 목록: {'해태포키블루베리41G': 0, '꼬깔콘고소한맛72G': 1, '농심오징어집83G': 2, '농심매운새우깡90G': 3, '크라운)콘초66G': 4, '농심바나나킥75G': 5, '롯데)자일리톨베타비타D용기86G': 6}
XML to YOLO 변환 및 데이터 분할 완료!


# a

In [2]:
from pathlib import Path

labels_all = Path('dataset/labels_all')

print(f"labels_all 폴더 내 파일 수: {len(list(labels_all.glob('*.txt')))}")
print("몇 개의 라벨 파일 예시:")
for label_file in list(labels_all.glob('*.txt'))[:5]:
    print(label_file.name)


labels_all 폴더 내 파일 수: 798
몇 개의 라벨 파일 예시:
10060_0_m_1.txt
10060_0_m_10.txt
10060_0_m_11.txt
10060_0_m_12.txt
10060_0_m_13.txt


# b

In [3]:
labels_train = Path('dataset/labels/train')
labels_val = Path('dataset/labels/val')

print(f"labels/train 폴더 내 파일 수: {len(list(labels_train.glob('*.txt')))}")
print(f"labels/val 폴더 내 파일 수: {len(list(labels_val.glob('*.txt')))}")

print("labels/train 폴더의 일부 파일:")
for label_file in list(labels_train.glob('*.txt'))[:5]:
    print(label_file.name)

print("labels/val 폴더의 일부 파일:")
for label_file in list(labels_val.glob('*.txt'))[:5]:
    print(label_file.name)


labels/train 폴더 내 파일 수: 760
labels/val 폴더 내 파일 수: 282
labels/train 폴더의 일부 파일:
10060_0_m_1.txt
10060_0_m_10.txt
10060_0_m_11.txt
10060_0_m_13.txt
10060_0_m_14.txt
labels/val 폴더의 일부 파일:
10060_0_m_1.txt
10060_0_m_10.txt
10060_0_m_11.txt
10060_0_m_12.txt
10060_0_m_13.txt


# c

In [4]:
sample_label = labels_train / '10060_0_m_1.txt'
if sample_label.exists():
    with open(sample_label, 'r') as f:
        content = f.read()
    print(f"{sample_label.name} 내용:\n{content}")
else:
    print(f"{sample_label.name} 파일이 존재하지 않습니다.")


10060_0_m_1.txt 내용:



# 3. 데이터 구성 파일 작성 (dataset.yaml)

In [6]:
import yaml

# dataset.yaml 파일 내용 생성
dataset_yaml = {
    'path': str(dataset_folder.resolve()),  # dataset 폴더의 절대 경로
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(classes),  # 클래스 수
    'names': classes  # 클래스 이름 리스트
}

# dataset.yaml 파일 저장
with open(project_root / 'dataset.yaml', 'w') as file:
    yaml.dump(dataset_yaml, file)

print("dataset.yaml 파일 생성 완료!")


dataset.yaml 파일 생성 완료!


# 4. YOLOv8 모델 학습

In [7]:
from ultralytics import YOLO

# 모델 로드 (YOLOv8n: 경량 모델)
model = YOLO('yolov8n.pt')  # yolov8n.pt, yolov8s.pt, yolov8m.pt 등으로 변경 가능

# 모델 학습
model.train(data='dataset.yaml', epochs=50, imgsz=640)


New https://pypi.org/project/ultralytics/8.3.22 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.21  Python-3.12.3 torch-2.4.1+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

train: Scanning C:\Users\Owner\OneDrive\바탕 화면\workspace\Snack\dataset\labels\train... 630 images, 111 backgrounds, 2 corrupt: 100%|██████████| 632/632 [00:00<00:00, 1596.19it/s]

train: WARNING  C:\Users\Owner\OneDrive\ \workspace\Snack\dataset\images\train\10091_60_s_18.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'C:\\Users\\Owner\\OneDrive\\ \\workspace\\Snack\\dataset\\images\\train\\10091_60_s_18.jpg'
train: WARNING  C:\Users\Owner\OneDrive\ \workspace\Snack\dataset\images\train\10091_60_s_19.jpg: ignoring corrupt image/label: cannot identify image file 'C:\\Users\\Owner\\OneDrive\\ \\workspace\\Snack\\dataset\\images\\train\\10091_60_s_19.jpg'


train: New cache created: C:\Users\Owner\OneDrive\ \workspace\Snack\dataset\labels\train.cache


val: Scanning C:\Users\Owner\OneDrive\바탕 화면\workspace\Snack\dataset\labels\val... 376 images, 216 backgrounds, 0 corrupt: 100%|██████████| 376/376 [00:00<00:00, 1803.85it/s]


val: New cache created: C:\Users\Owner\OneDrive\ \workspace\Snack\dataset\labels\val.cache
Plotting labels to runs\detect\train3\labels.jpg... 


: 

# 모델 검증

In [ ]:
# 모델 검증
results = model.val()
print(results)


# 5. 학습된 모델 검증 및 사용

In [ ]:
# 모델 검증
results = model.val()
print(results)

# 예측 수행할 이미지 폴더 경로 (상대 경로 사용)
test_images = image_folder / '10060_해태포키블루베리41G'  # 예시: 특정 클래스 폴더

# 객체 감지 수행
results = model.predict(source=str(test_images), save=True)  # save=True: 결과 이미지 저장

# 결과 확인
for r in results:
    r.plot()  # 결과 이미지 표시


# 객체 감지 실행

In [ ]:
# 예측 수행할 이미지 폴더 경로 (상대 경로 사용)
test_images = image_folder / '10060_해태포키블루베리41G'  # 예시: 특정 클래스 폴더

# 객체 감지 수행
results = model.predict(source=str(test_images), save=True)  # save=True: 결과 이미지 저장

# 결과 확인
for r in results:
    r.plot()  # 결과 이미지 표시
